# Installing Dataset And Packages

### Google Collab Link

https://colab.research.google.com/drive/128sBCKkm0L2nvtPbiV83cBh4KSXTlI9s?usp=sharing

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset
import torch
import os

# Training And Finetuning Phi-2 Model using LORA


### Used Google T-4 GPU

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# Load and prepare dataset
print("Loading dataset...")
data = load_dataset('gretelai/synthetic_text_to_sql')
data = data.filter(lambda row: row['domain'] == 'financial services' and row['sql_task_type'] == 'analytics and reporting')
data = data.select_columns(['sql_prompt', 'sql_context', 'sql'])

# Prompt template
PROMPT_TEMPLATE = """Instruct: Generate a ClickHouse SQL query to answer: `{prompt}`

Schema:
{system}

Response:
```sql
"""

def format_text(row):
    return {
        'text': PROMPT_TEMPLATE.format(
            prompt=row['sql_prompt'],
            system=row['sql_context']
        ) + row['sql'] + "\n```"
    }

print("Formatting data...")
data = data.map(format_text, remove_columns=['sql_prompt', 'sql_context', 'sql'])

# Quantization config
print("Setting up quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load tokenizer
print("Loading tokenizer...")
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Load model
print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto"
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA config
print("Preparing LoRA...")
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Tokenization
print("Tokenizing data...")
def tokenize(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )

tokenized_data = data.map(tokenize, batched=True, remove_columns=['text'])

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Training arguments
print("Setting up training...")
training_args = TrainingArguments(
    output_dir="./phi2-sql-finetuned",
    run_name="phi2-sql-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    gradient_checkpointing=True,
    optim="adafactor",
    remove_unused_columns=False,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    report_to="wandb"  # Disable wandb if not needed
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    data_collator=data_collator,
)

# Train
print("Starting training...")
trainer.train()

# Save
print("Saving model...")
model.save_pretrained("./phi2-sql-finetuned")
tokenizer.save_pretrained("./phi2-sql-finetuned")

print("Training complete!")

Loading dataset...
Formatting data...
Setting up quantization...
Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Preparing LoRA...
trainable params: 5,242,880 || all params: 2,784,926,720 || trainable%: 0.1883
Tokenizing data...


Map:   0%|          | 0/66 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Setting up training...
Starting training...


Epoch,Training Loss,Validation Loss
1,0.588100,0.523918
2,0.429900,0.404543


Saving model...
Training complete!


### Both Training And Validation Losses Seems To Have Reduced, Showcasing Model Is Learning Well

# Generation, Optimization, Validation [Precision, Recall, F1-Score]

In [ ]:
import re
import sqlparse
from typing import Dict, List
from difflib import SequenceMatcher
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

class TextToSQLSystem:
    def __init__(self, model_path="./phi2-sql-finetuned"):
        # Load model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        base_model = AutoModelForCausalLM.from_pretrained(
            "microsoft/phi-2",
            trust_remote_code=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
        self.model = PeftModel.from_pretrained(base_model, model_path)
        self.model.eval()

    def generate_sql(self, prompt: str, schema: str) -> Dict:
        """Generate and optimize SQL query"""
        try:
            # Generate SQL
            input_text = f"""Instruct: Generate a ClickHouse SQL query to answer: `{prompt}`

Schema:
{schema}

Response:
```sql
"""
            inputs = self.tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to("cuda")

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=256,
                    temperature=0.3,
                    do_sample=True,
                    top_p=0.9
                )

            # Extract SQL
            full_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            sql_match = re.search(r'```sql\n(.*?)\n```', full_output, re.DOTALL)
            raw_sql = sql_match.group(1).strip() if sql_match else full_output.split("```sql")[-1].split("```")[0].strip()

            # Optimize and validate
            formatted_sql = sqlparse.format(raw_sql, reindent=True, keyword_case='upper')
            analysis = self._analyze_query(raw_sql)

            return {
                'success': True,
                'sql': raw_sql,
                'formatted_sql': formatted_sql,
                'optimizations': self._suggest_optimizations(analysis),
                'warnings': self._validate_sql(raw_sql)['warnings']
            }

        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'sql': ""
            }

    def _analyze_query(self, sql: str) -> Dict:
        """Analyze SQL query structure"""
        parsed = sqlparse.parse(sql)[0]
        analysis = {
            'tables': set(),
            'joins': 0,
            'has_where': False,
            'has_limit': False
        }

        for token in parsed.tokens:
            if token.ttype is sqlparse.tokens.DML and token.value.upper() == 'SELECT':
                for identifier in parsed.get_sublists():
                    if isinstance(identifier, sqlparse.sql.IdentifierList):
                        for item in identifier.get_identifiers():
                            analysis['tables'].add(item.get_real_name())
                    elif isinstance(identifier, sqlparse.sql.Identifier):
                        analysis['tables'].add(identifier.get_real_name())

            if token.ttype is sqlparse.tokens.Keyword and token.value.upper() == 'JOIN':
                analysis['joins'] += 1
            elif token.value.upper() == 'WHERE':
                analysis['has_where'] = True
            elif token.value.upper() == 'LIMIT':
                analysis['has_limit'] = True

        return analysis

    def _suggest_optimizations(self, analysis: Dict) -> List[str]:
        """Generate optimization suggestions"""
        suggestions = []
        if len(analysis['tables']) > 3:
            suggestions.append("Consider denormalizing data for queries joining many tables")
        if analysis['joins'] > 2:
            suggestions.append("Ensure proper indexes exist on join columns")
        if not analysis['has_where']:
            suggestions.append("Add WHERE clause to avoid full table scans")
        if not analysis['has_limit']:
            suggestions.append("Consider adding LIMIT to prevent large result sets")
        return suggestions

    def _validate_sql(self, sql: str) -> Dict:
        """Validate SQL syntax"""
        validation = {'valid': True, 'warnings': []}
        sql_lower = sql.lower()

        if not any(cmd in sql_lower for cmd in ['select', 'insert', 'update']):
            validation['warnings'].append("Query may be missing a main SQL command")
        if 'drop' in sql_lower or 'delete' in sql_lower:
            validation['warnings'].append("Query contains potentially dangerous operation")
        return validation

class SQLEvaluator:
    @staticmethod
    def evaluate(predicted: str, reference: str) -> Dict:
        """Calculate precision, recall, and F1 score"""
        def tokenize(sql):
            sql = re.sub(r'--.*?\n', ' ', sql.lower())
            sql = re.sub(r'/\*.*?\*/', ' ', sql, flags=re.DOTALL)
            return [t for t in re.findall(r'(\b\w+\b|[,;()=<>+*/])', ' '.join(sql.split())) if t.strip()]

        pred_tokens = tokenize(predicted)
        ref_tokens = tokenize(reference)

        # Create binary vectors
        all_tokens = list(set(pred_tokens + ref_tokens))
        token_to_idx = {t: i for i, t in enumerate(all_tokens)}

        pred_vec = np.zeros(len(all_tokens))
        ref_vec = np.zeros(len(all_tokens))

        for t in pred_tokens:
            pred_vec[token_to_idx[t]] = 1
        for t in ref_tokens:
            ref_vec[token_to_idx[t]] = 1

        return {
            'precision': precision_score(ref_vec, pred_vec, zero_division=0),
            'recall': recall_score(ref_vec, pred_vec, zero_division=0),
            'f1': f1_score(ref_vec, pred_vec, zero_division=0),
            'similarity': SequenceMatcher(None, pred_tokens, ref_tokens).ratio(),
            'exact_match': predicted.lower().strip() == reference.lower().strip()
        }

# Example usage
if __name__ == "__main__":
    # Initialize system
    sql_system = TextToSQLSystem()

    # Example schema and prompt
    schema = """
    CREATE TABLE accounts (
        account_id INT PRIMARY KEY,
        customer_id INT,
        balance DECIMAL(15,2),
        account_type VARCHAR(20)
    );
    CREATE TABLE transactions (
        transaction_id INT PRIMARY KEY,
        account_id INT,
        amount DECIMAL(15,2),
        date DATE
    );
    """

    prompt = "Show me all transactions over $1000 for checking accounts last month"

    # Generate SQL
    result = sql_system.generate_sql(prompt, schema)
    print("Generated SQL:")
    print(result['formatted_sql'])
    print("\nOptimizations:")
    for opt in result['optimizations']:
        print(f"- {opt}")

    # Evaluation example
    reference_sql = "SELECT * FROM transactions WHERE amount > 1000 AND date >= '2023-11-01'"
    metrics = SQLEvaluator.evaluate(result['sql'], reference_sql)
    print("\nEvaluation Metrics:")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")
    print(f"Similarity: {metrics['similarity']:.2f}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated SQL:
SELECT *
FROM transactions
WHERE account_type = 'checking'
  AND amount > 1000
  AND date BETWEEN DATE_SUB(CURDATE(), INTERVAL 1 MONTH) AND CURDATE();

Optimizations:
- Add WHERE clause to avoid full table scans
- Consider adding LIMIT to prevent large result sets

Evaluation Metrics:
Precision: 0.48
Recall: 0.79
F1 Score: 0.59
Similarity: 0.44
